# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('D:\Azzam\Personal_Projects\SDA\OpenAI_API\Langchain\lab-chains-in-langchain\data\Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [ ]:
pip install langchain_community

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.3)
# I chose temperature=0.3 to ensure that the model generates consistent and reliable responses

In [9]:
prompt = ChatPromptTemplate.from_template( #Write a query that would take a variable to describe any product
    "Can you describe the product named '{product}' based on customer reviews?"
)

In [10]:
chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\azams\AppData\Local\Temp\ipykernel_3476\1305865249.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [11]:
product = "Luxury Air Mattress"
chain.run(product)

C:\Users\azams\AppData\Local\Temp\ipykernel_3476\1892552926.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


"Based on customer reviews, the Luxury Air Mattress is a high-quality and comfortable sleeping solution that provides a luxurious experience. Customers have praised the mattress for its durability, support, and overall comfort. Many have mentioned that the mattress is easy to inflate and deflate, and holds air well throughout the night. The plush surface of the mattress provides a soft and cozy sleeping surface, while the sturdy construction ensures a stable and supportive night's rest. Overall, customers have been highly satisfied with the Luxury Air Mattress, describing it as a top-of-the-line product that offers a superior sleeping experience."

## SimpleSequentialChain

In [12]:
from langchain.chains import SimpleSequentialChain

In [13]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    #Repeat the initial query or create a new query that would feed into the second prompt
    "Describe the product called '{product}' based on customer reviews."
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [14]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    #Write the second prompt query that takes an input variable whose input will come from the previous prompt"
    "Based on the following description, write a short customer testimonial:\n\n{text}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [15]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [16]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Based on customer reviews, the Luxury Air Mattress is a top-of-the-line product that provides an incredibly comfortable and luxurious sleeping experience. Customers rave about the plushness and support of the mattress, noting that it feels just like sleeping on a traditional bed. Many customers also praise the durability and longevity of the mattress, stating that it holds up well over time and maintains its shape and comfort level.

Customers appreciate the convenient features of the Luxury Air Mattress, such as the built-in pump for easy inflation and deflation, as well as the included carrying bag for portability and storage. Many customers also comment on the impressive size and height of the mattress, noting that it provides ample space and a raised surface that feels similar to a regular bed.

Overall, customers highly recommend the Luxury Air Mattress for those seeking a comfortable and high-quality sleeping option for their home o

"I absolutely love my Luxury Air Mattress! It feels just like sleeping on a traditional bed, and the plushness and support are incredible. The built-in pump and carrying bag make it so convenient to use and transport, and I really appreciate the durability and longevity of the mattress. It's definitely a top-of-the-line product that I would recommend to anyone looking for a comfortable and luxurious sleeping experience."

**Repeat the above twice for different products**

## SequentialChain

In [17]:
from langchain.chains import SequentialChain

In [18]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  #This prompt should translate a review
  "Translate the following product review to English:\n\n{review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="english_review" #Give a name to your output
                    )


In [19]:
second_prompt = ChatPromptTemplate.from_template(
    #Write a promplt to summarize a review
    "Summarize the following English product review:\n\n{english_review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary" #give a name to this output
                    )


In [20]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "What is the language of the following review?\n\n{review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [21]:
# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "The review was originally written in {language}. Based on the summary below, write a short follow-up message to the customer:\n\n{summary}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [22]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["english_review", "summary", "language", "followup_message"],
    verbose=True
)

In [23]:
review = df.Review[5]
overall_chain(review)

C:\Users\azams\AppData\Local\Temp\ipykernel_3476\1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'english_review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is not impressed with the taste and consistency of the product, suggesting that it may be an old batch or counterfeit compared to the ones purchased in stores which they find to have a much better taste.',
 'language': 'French',
 'followup_message': "Bonjour,\n\nNous sommes désolés d'apprendre que vous n'avez pas apprécié le goût et la consistance du produit que vous avez reçu. Nous vous assurons que nous prenons la qualité de nos produits très au sérieux et nous allons enquêter sur cette situation pour nous assurer que nos clients reçoivent toujours des produits de la plus haute qualité. Merci de nous avoir inf

**Repeat the above twice for different products or reviews**

In [27]:
review = df.Review[6]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': 'Está lu bonita calienta muy rápido, es muy funcional, solo falta ver cuánto dura, solo llevo 3 días en funcionamiento.',
 'english_review': 'It is very pretty, heats up very quickly, it is very functional, just need to see how long it lasts, I have only been using it for 3 days.',
 'summary': 'The reviewer finds the product attractive and efficient, noting that it heats up quickly and is functional. They are unsure of its longevity as they have only been using it for three days.',
 'language': 'Spanish',
 'followup_message': "Hi [Customer],\n\nThank you for your positive feedback on our product! We're glad to hear that you find it attractive and efficient. If you have any concerns about the longevity of the product after using it for a longer period of time, please don't hesitate to reach out to us. We want to ensure your complete satisfaction with your purchase.\n\nBest regards,\n[Your Name]"}

In [28]:
review = df.Review[3]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': 'This is the best throw pillow fillers on Amazon. I’ve tried several others, and they’re all cheap and flat no matter how much fluffing you do. Once you toss these in the dryer after you remove them from the vacuum sealed shipping material, they fluff up great',
 'english_review': 'This is the best throw pillow fillers on Amazon. I’ve tried several others, and they’re all cheap and flat no matter how much fluffing you do. Once you toss these in the dryer after you remove them from the vacuum sealed shipping material, they fluff up great.',
 'summary': 'The reviewer claims that these throw pillow fillers are the best on Amazon, as they are fluffier than others they have tried. They recommend putting them in the dryer after removing them from the vacuum sealed packaging for best results.',
 'language': 'English',
 'followup_message': 'Dear customer,\n\nThank you for leaving such a positive review of our throw pillow fillers! We are thrilled to hear that you found them to be th

## Router Chain

In [31]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [32]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [33]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [34]:
llm = ChatOpenAI(temperature=0)

In [35]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [36]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [37]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [38]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [39]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\azams\AppData\Local\Temp\ipykernel_3476\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [40]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics."

In [41]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 is equal to 4.'

In [42]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures the proper functioning of the organism as a whole.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [43]:
chain.run("How do plants perform photosynthesis?")



> Entering new MultiPromptChain chain...
biology: {'input': 'How do plants perform photosynthesis?'}
> Finished chain.


'Plants perform photosynthesis by using sunlight, carbon dioxide, and water to produce glucose (sugar) and oxygen. This process takes place in the chloroplasts of plant cells, specifically in the thylakoid membranes. \n\nDuring photosynthesis, sunlight is absorbed by chlorophyll, a pigment in the chloroplasts, and this energy is used to split water molecules into oxygen and hydrogen ions. The oxygen is released as a byproduct, while the hydrogen ions are used to generate ATP (adenosine triphosphate) and NADPH (nicotinamide adenine dinucleotide phosphate), which are energy carriers.\n\nThe ATP and NADPH are then used in the Calvin cycle, a series of biochemical reactions that convert carbon dioxide into glucose. This process involves a series of enzyme-catalyzed reactions that ultimately produce glucose, which can be used by the plant for energy and growth.\n\nOverall, photosynthesis is a crucial process for plants as it provides them with the energy they need to survive and grow, while

In [44]:
chain.run("What were the main causes of World War I?")



> Entering new MultiPromptChain chain...
History: {'input': 'What were the main causes of World War I?'}
> Finished chain.


"There were several main causes of World War I, including:\n\n1. Nationalism: The intense pride and loyalty to one's own nation led to competition and rivalries between countries, particularly in Europe.\n\n2. Imperialism: European powers were competing for colonies and resources around the world, leading to tensions and conflicts.\n\n3. Militarism: The arms race between countries, particularly in Europe, led to an increase in military spending and the development of powerful armies and navies.\n\n4. Alliances: The system of alliances between countries created a complex web of obligations and commitments that could easily escalate conflicts.\n\n5. Assassination of Archduke Franz Ferdinand: The assassination of the heir to the Austro-Hungarian throne in Sarajevo in 1914 was the immediate trigger for the outbreak of war.\n\nThese factors, combined with long-standing tensions and rivalries between European powers, ultimately led to the outbreak of World War I in 1914."

In [45]:
chain.run("How does a binary search algorithm work?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'How does a binary search algorithm work?'}
> Finished chain.


'A binary search algorithm works by repeatedly dividing the search interval in half. It starts by comparing the target value to the middle element of the array. If the target value is equal to the middle element, the search is successful. If the target value is less than the middle element, the algorithm continues the search on the lower half of the array. If the target value is greater than the middle element, the algorithm continues the search on the upper half of the array.\n\nThis process is repeated until the target value is found or the search interval is empty. Binary search is efficient because it eliminates half of the remaining elements in each iteration, resulting in a time complexity of O(log n), where n is the number of elements in the array. Additionally, binary search is a space-efficient algorithm as it only requires a constant amount of additional space for storing the indices of the search interval.'

In [46]:
chain.run("Can you explain the Pythagorean theorem?")



> Entering new MultiPromptChain chain...
math: {'input': 'Can you explain the Pythagorean theorem?'}
> Finished chain.


'Of course! The Pythagorean theorem is a fundamental principle in geometry that relates to right-angled triangles. It states that in a right triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides.\n\nIn mathematical terms, if a and b are the lengths of the two shorter sides of a right triangle, and c is the length of the hypotenuse, then the Pythagorean theorem can be written as:\n\na^2 + b^2 = c^2\n\nThis theorem is named after the ancient Greek mathematician Pythagoras, who is credited with its discovery. The Pythagorean theorem is not only important for solving problems in geometry, but it also has applications in various fields such as physics, engineering, and computer science.'